# 케라스 콜백

## ModelCheckpoint 와 EarlyStopping 콜백

In [1]:
import keras

In [ ]:
callbacks_list = [
    keras.callbacks.EarlyStopping( # 성능 향상이 멈추면 훈련 중지
        monitor='val_acc',
        patience=1, # 1 epoch보다 더 길게 (즉 2 epoch 동안) 정확도가 향상되지 않으면 훈련이 중지됨.
    ),
    keras.callbacks.ModelCheckpoint( # epoch마다 현재 가중치 저장
        filepath='my_model.h5',
        monitor='val_loss', # 아래 두 매개변수는 val_loss가 좋아지지 않으면 모델 파일을 덮어쓰지 않는다는 뜻. 훈련하는 동안 가장 좋은 모델이 저장됨.
        save_best_only=True,
    )
]

model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['acc'])

model.fit(x,y,
         epochs=10,
         batch_size=32,
         callbacks=callbacks_list,
         validation_data=(x_val, y_val))

## ReduceLROnPlateau 콜백

In [ ]:
callbacks_list = [
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1, # 콜백이 호출될 떄 학습률을 10배로 줄임.
        patience=10, # 검증 손실이 10 epoch 동안 좋아지지 않으면 콜백이 호출됨.
    )
]

model.fit(x, y,
         epochs=10,
         batch_size=32,
         callbacks=callbacks_list,
         validation_data=(x_val, y_val))

## 자신만의 콜백 만들기

In [ ]:
# 매 에포크의 끝에서 검증 세트의 첫 번째 샘플로 모델에 있는 모든 층의 활성화 출력을 계산하여 (넘파이 배열로) 디스크에 저장하는 자작 콜백의 예

class ActivationLogger(keras.callbacks.Callback):
    def set_model(self, model): # 호출하는 모델에 대한 정보를 전달하기 위해 훈련하기 전에 호출됨.
        self.model = model
        layer_outputs = [layer.output for layer in model.layers]
        self.activations_model = keras.models.Model(model.input, layer_outputs) # 각 층의 활성화 출력을 반환하는 Model 객체
        
    def on_epoch_end(self, epoch, logs=None):
        if self.validation_data is None:
            raise RuntimeError('Requires validation_data.')
            
        validation_sample = self.validation_data[0][0:1]
        activations = self.activations_model.predict(validation_sample)
        f = open('activation_at_epoch_'+str(epoch)+'.npz','wb')
        np.savez(f, activations)
        f.close()